In [1]:
from delta import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
builder = (SparkSession.builder
           .appName("monitor-stream")
           .master("spark://spark-master:7077")
           .config("spark.executor.memory", "512m")
           .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
           .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog"))

spark = configure_spark_with_delta_pip(builder,['org.apache.spark:spark-sql-kafka-0-10_2.12:3.4.1']).getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-62083e88-2eea-4a25-9a17-575f3e003f3b;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.4.0 in central
	found io.delta#delta-storage;2.4.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.4.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.4.1 in central
	found org.apache.kafka#kafka-clients;3.3.2 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.1 in central
	found org.slf4j#slf4j-api;2.0.6 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in centra

In [3]:
get_ipython().run_line_magic('load_ext', 'sparksql_magic')
get_ipython().run_line_magic('config', 'SparkSql.limit=20')

In [4]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

schema = StructType([
    StructField('id', IntegerType(), True),
    StructField('name', StringType(), True),
    StructField('age', IntegerType(), True),
    StructField('gender', StringType(), True),
    StructField('country', StringType(), True),
    StructField('timestamp', StringType(), True)])

users_df = (spark.readStream
      .format("kafka")
      .option("kafka.bootstrap.servers", "kafka:9092")
      .option("subscribe", "users")
      .option("startingOffsets", "latest")
      .load()
      .withColumn('value', from_json(col('value').cast("STRING"), schema)))

users_df = users_df.select(
    col('value.id').alias('id'),
    col('value.name').alias('name'),
    col('value.age').alias('age'),
    col('value.gender').alias('gender'),
    col('value.country').alias('country'),
    to_timestamp(col('value.timestamp'), "MM/dd/yyyy, HH:mm:ss").alias('timestamp'))

In [5]:
query = (users_df.writeStream
   .format("delta")
   .queryName("user-kafka-stream")
   .outputMode("append")
   .option("checkpointLocation", "/opt/workspace/data/delta_lake/monitor-streams/users/_checkpoints/")
   .start("/opt/workspace/data/delta_lake/monitor-streams/users"))

In [6]:
query.status

{'message': 'Processing new data',
 'isDataAvailable': True,
 'isTriggerActive': True}

In [7]:
query.recentProgress

[{'id': '44996f2d-15c6-49d2-9185-72fd08ab7132',
  'runId': '8b493789-ebd9-4e40-afc8-26094de17eb3',
  'name': 'user-kafka-stream',
  'timestamp': '2023-08-26T12:32:12.028Z',
  'batchId': 0,
  'numInputRows': 0,
  'inputRowsPerSecond': 0.0,
  'processedRowsPerSecond': 0.0,
  'durationMs': {'addBatch': 20871,
   'commitOffsets': 162,
   'getBatch': 28,
   'latestOffset': 997,
   'queryPlanning': 506,
   'triggerExecution': 22715,
   'walCommit': 116},
  'stateOperators': [],
  'sources': [{'description': 'KafkaV2[Subscribe[users]]',
    'startOffset': None,
    'endOffset': {'users': {'0': 310}},
    'latestOffset': {'users': {'0': 310}},
    'numInputRows': 0,
    'inputRowsPerSecond': 0.0,
    'processedRowsPerSecond': 0.0,
    'metrics': {'avgOffsetsBehindLatest': '0.0',
     'maxOffsetsBehindLatest': '0',
     'minOffsetsBehindLatest': '0'}}],
  'sink': {'description': 'DeltaSink[/opt/workspace/data/delta_lake/monitor-streams/users]',
   'numOutputRows': -1}}]

In [8]:
from pyspark.sql.streaming import StreamingQueryListener

# Define a custom listener class
class MyListener(StreamingQueryListener):

    # Override the onQueryStarted method
    def onQueryStarted(self, event):
        # Print the query name and id when it starts
        print(f"'{event.name}' [{event.id}] got started!")

    # Override the onQueryProgress method
    def onQueryProgress(self, event):
        # Print the input rate and processing rate when it progresses
        print(f"Query made progress: " + str(event.progress))

    # Override the onQueryTerminated method
    def onQueryTerminated(self, event):
        # Print the exception message when it terminates
        if event.exception:
            print(f"Query with id {event.id} terminated with exception: {event}")
        else:
            print(f"Query with id {event.id} terminated normally")

# Create an instance of the listener class
listener = MyListener()

# Register the listener with spark.streams
spark.streams.addListener(listener)

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:33:42.109Z",
  "batchId" : 8,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 71.42857142857143,
  "processedRowsPerSecond" : 0.33377837116154874,
  "durationMs" : {
    "addBatch" : 2774,
    "commitOffsets" : 110,
    "getBatch" : 0,
    "latestOffset" : 3,
    "queryPlanning" : 9,
    "triggerExecution" : 2996,
    "walCommit" : 99
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 318
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 319
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 319
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 71.42857142857143,
    "processedRowsPerSecond" : 0.33377837116154874,
    "metrics" : {
      "avgOffsetsBehi

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:33:52.129Z",
  "batchId" : 9,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 71.42857142857143,
  "processedRowsPerSecond" : 0.30450669914738127,
  "durationMs" : {
    "addBatch" : 3002,
    "commitOffsets" : 135,
    "getBatch" : 0,
    "latestOffset" : 4,
    "queryPlanning" : 17,
    "triggerExecution" : 3284,
    "walCommit" : 124
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 319
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 320
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 320
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 71.42857142857143,
    "processedRowsPerSecond" : 0.30450669914738127,
    "metrics" : {
      "avgOffsetsBe

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:34:02.135Z",
  "batchId" : 10,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 71.42857142857143,
  "processedRowsPerSecond" : 0.1592610288262462,
  "durationMs" : {
    "addBatch" : 6060,
    "commitOffsets" : 113,
    "getBatch" : 0,
    "latestOffset" : 4,
    "queryPlanning" : 8,
    "triggerExecution" : 6279,
    "walCommit" : 92
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 320
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 321
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 321
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 71.42857142857143,
    "processedRowsPerSecond" : 0.1592610288262462,
    "metrics" : {
      "avgOffsetsBehin

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:34:12.145Z",
  "batchId" : 11,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 76.92307692307692,
  "processedRowsPerSecond" : 0.1272264631043257,
  "durationMs" : {
    "addBatch" : 7608,
    "commitOffsets" : 150,
    "getBatch" : 0,
    "latestOffset" : 2,
    "queryPlanning" : 12,
    "triggerExecution" : 7860,
    "walCommit" : 87
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 321
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 322
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 322
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 76.92307692307692,
    "processedRowsPerSecond" : 0.1272264631043257,
    "metrics" : {
      "avgOffsetsBehi

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:34:22.154Z",
  "batchId" : 12,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 76.92307692307692,
  "processedRowsPerSecond" : 0.3281916639317361,
  "durationMs" : {
    "addBatch" : 2816,
    "commitOffsets" : 120,
    "getBatch" : 0,
    "latestOffset" : 2,
    "queryPlanning" : 9,
    "triggerExecution" : 3047,
    "walCommit" : 99
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 322
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 323
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 323
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 76.92307692307692,
    "processedRowsPerSecond" : 0.3281916639317361,
    "metrics" : {
      "avgOffsetsBehin

...

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:35:22.211Z",
  "batchId" : 18,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 76.92307692307692,
  "processedRowsPerSecond" : 0.35650623885918004,
  "durationMs" : {
    "addBatch" : 2600,
    "commitOffsets" : 105,
    "getBatch" : 0,
    "latestOffset" : 2,
    "queryPlanning" : 8,
    "triggerExecution" : 2805,
    "walCommit" : 89
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 328
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 329
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 329
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 76.92307692307692,
    "processedRowsPerSecond" : 0.35650623885918004,
    "metrics" : {
      "avgOffsetsBeh

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:35:32.217Z",
  "batchId" : 19,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 83.33333333333333,
  "processedRowsPerSecond" : 0.2934272300469484,
  "durationMs" : {
    "addBatch" : 3156,
    "commitOffsets" : 137,
    "getBatch" : 0,
    "latestOffset" : 2,
    "queryPlanning" : 8,
    "triggerExecution" : 3408,
    "walCommit" : 103
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 329
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 330
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 330
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 83.33333333333333,
    "processedRowsPerSecond" : 0.2934272300469484,
    "metrics" : {
      "avgOffsetsBehi

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:35:42.222Z",
  "batchId" : 20,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 71.42857142857143,
  "processedRowsPerSecond" : 0.2684563758389262,
  "durationMs" : {
    "addBatch" : 3538,
    "commitOffsets" : 88,
    "getBatch" : 0,
    "latestOffset" : 3,
    "queryPlanning" : 8,
    "triggerExecution" : 3725,
    "walCommit" : 88
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 330
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 331
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 331
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 71.42857142857143,
    "processedRowsPerSecond" : 0.2684563758389262,
    "metrics" : {
      "avgOffsetsBehind

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:35:52.236Z",
  "batchId" : 21,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 76.92307692307692,
  "processedRowsPerSecond" : 0.21800741225201659,
  "durationMs" : {
    "addBatch" : 4347,
    "commitOffsets" : 129,
    "getBatch" : 0,
    "latestOffset" : 2,
    "queryPlanning" : 9,
    "triggerExecution" : 4587,
    "walCommit" : 99
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 331
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 332
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 332
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 76.92307692307692,
    "processedRowsPerSecond" : 0.21800741225201659,
    "metrics" : {
      "avgOffsetsBeh

Query made progress: {
  "id" : "44996f2d-15c6-49d2-9185-72fd08ab7132",
  "runId" : "8b493789-ebd9-4e40-afc8-26094de17eb3",
  "name" : "user-kafka-stream",
  "timestamp" : "2023-08-26T12:36:02.243Z",
  "batchId" : 22,
  "numInputRows" : 1,
  "inputRowsPerSecond" : 83.33333333333333,
  "processedRowsPerSecond" : 0.363901018922853,
  "durationMs" : {
    "addBatch" : 2559,
    "commitOffsets" : 87,
    "getBatch" : 0,
    "latestOffset" : 2,
    "queryPlanning" : 9,
    "triggerExecution" : 2748,
    "walCommit" : 91
  },
  "stateOperators" : [ ],
  "sources" : [ {
    "description" : "KafkaV2[Subscribe[users]]",
    "startOffset" : {
      "users" : {
        "0" : 332
      }
    },
    "endOffset" : {
      "users" : {
        "0" : 333
      }
    },
    "latestOffset" : {
      "users" : {
        "0" : 333
      }
    },
    "numInputRows" : 1,
    "inputRowsPerSecond" : 83.33333333333333,
    "processedRowsPerSecond" : 0.363901018922853,
    "metrics" : {
      "avgOffsetsBehindLa

In [9]:
query.stop()

Query with id 44996f2d-15c6-49d2-9185-72fd08ab7132 terminated normally


In [10]:
spark.stop() 